In [1]:
import cv2

# カスケードファイル「haarcascade_frontalface_alt.xml」を指定
eye_cascade_file = 'haarcascade_eye.xml'
# 分類機を作成 
eye_cascade = cv2.CascadeClassifier(eye_cascade_file)

cap = cv2.VideoCapture(0)
img_last = None # 前回の画像を記憶する変数
# 線の色の定義
blue = (255, 0, 0)

while True:
    # 画像を取得
    _, frame = cap.read()
    frame = cv2.resize(frame, (500, 300))
    
    # 白黒画像に変換
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (9, 9), 0)
    img_b = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY)[1]
    
    # 目の検出を実行
    eyes = eye_cascade.detectMultiScale(gray, minSize=(50, 50))
    
    # 差分を確認する
    if img_last is None:
        img_last = img_b
        continue
    frame_diff = cv2.absdiff(img_last, img_b)
    cnts = cv2.findContours(frame_diff, 
            cv2.RETR_EXTERNAL,
            cv2.CHAIN_APPROX_SIMPLE)[0]
    
    # 差分があった点を画面に描く
    for x, y, w, h in eyes:
        cv2.rectangle(frame, (x, y), (x+w, y+h), blue, thickness=5)
        
    # 今回のフレームを保存
    img_last = img_b
    # 画面に表示
    #(２画面表示されます！！)
    cv2.imshow("Diff Camera", frame)
    if cv2.waitKey(1) == 13: break
cap.release()
cv2.destroyAllWindows()